In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import lxml
import html5lib

Now that necessary packages are installed, going to parse the needed url. This URL brings us to the up to date team page for the Winnipeg Jets . So I am going to scrape the cap friendly website (Team tab, WPG Jets) and get this into a couple different dataframes (Draft, Forwards, Defense, Goalies, Season Opening Injured Reserve) that I can download and put into the MySQL database.

In [2]:
url_v3 = "https://www.capfriendly.com/teams/goldenknights"

req2 = requests.get(url_v3)
soup = BeautifulSoup(req2.content, 'lxml')  # make a soup of html & css from the web page

In [3]:
#Find all tables
table = soup.find_all('table', class_='cf_teamProfileRosterSection__table cf_table__collapsiblePlayerColumn cf_table__default cf_table__fixed cf_table__stickyFirstColumn')

forwards_df = pd.read_html(str(table),header=0, index_col = 0, na_values=["-"])[0]

def conditional_splitx(value):
    if isinstance(value, str) and value.startswith('$'):
        # Split the value by '$' and keep the first part along with the first '$'
        first_dollar_index = value.find('$', 1)
        return value[:first_dollar_index]
    return value

# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    forwards_df[column] = forwards_df[column].apply(conditional_splitx)

forwards_df.to_csv('Cap Friendly VGK Forwards.csv')


C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\1484051297.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  forwards_df = pd.read_html(str(table),header=0, index_col = 0, na_values=["-"])[0]


Forwards are scraped. Now time to scrape defensemen.

In [4]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    defensemen_table = tables[2]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
df_defensemen = pd.read_html(str(defensemen_table))[0]

#Function to clean the defense table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    df_defensemen[column] = df_defensemen[column].apply(conditional_splitx)

df_defensemen.to_csv('Cap Friendly VGK Defense.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\3731938331.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_defensemen = pd.read_html(str(defensemen_table))[0]


Now that we have forwards and defense, time for goalies

In [5]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    goalies_table = tables[3]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
goalies_df = pd.read_html(str(goalies_table))[0]

#Function to clean the goalies table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    goalies_df[column] = goalies_df[column].apply(conditional_splitx)

goalies_df.to_csv('Cap Friendly VGK Goalies.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\616846663.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goalies_df = pd.read_html(str(goalies_table))[0]


Forwards, Defense, and Goalies Scraped. Now time for LTIR

In [6]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    LTIR_table = tables[4]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
LTIR_df = pd.read_html(str(LTIR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    LTIR_df[column] = LTIR_df[column].apply(conditional_splitx)

LTIR_df.to_csv('Cap Friendly VGK LTIR.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\2122527447.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  LTIR_df = pd.read_html(str(LTIR_table))[0]


Non-Roster Time. First up, Forwards

In [7]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    forwardsNR_table = tables[8]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
forwardsNR_df = pd.read_html(str(forwardsNR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    forwardsNR_df[column] = forwardsNR_df[column].apply(conditional_splitx)

forwardsNR_df.to_csv('Cap Friendly VGK Non-Roster Forwards.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\3692839390.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  forwardsNR_df = pd.read_html(str(forwardsNR_table))[0]


Non-Roster Defense

In [8]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    defenseNR_table = tables[9]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
defenseNR_df = pd.read_html(str(defenseNR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    defenseNR_df[column] = defenseNR_df[column].apply(conditional_splitx)

defenseNR_df.to_csv('Cap Friendly VGK Non-Roster Defense.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\2956773941.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defenseNR_df = pd.read_html(str(defenseNR_table))[0]


Non-Roster Goalies

In [9]:
tables = soup.find_all('table', class_=['cf_teamProfileRosterSection__table', 
                                        'cf_table__collapsiblePlayerColumn', 
                                        'cf_table__default', 
                                        'cf_table__fixed', 
                                        'cf_table__stickyFirstColumn'])

# Get the fifth instance of the table
if len(tables) > 4:
    goaliesNR_table = tables[10]  # index 4 for the fifth instance
else:
    raise ValueError("The specified table class does not have a fifth instance.")

# Convert the table to a DataFrame
goaliesNR_df = pd.read_html(str(goaliesNR_table))[0]

#Function to clean the season open IR table
# List of columns to apply the function to
columns_to_clean = ['2024-25', '2025-26','2026-27','2027-28','2028-29','2029-30']

# Apply the function to each column in the list
for column in columns_to_clean:
    goaliesNR_df[column] = goaliesNR_df[column].apply(conditional_splitx)

goaliesNR_df.to_csv('Cap Friendly VGK Non-Roster Goalies.csv')

C:\Users\nsofi\AppData\Local\Temp\ipykernel_37320\2309354399.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  goaliesNR_df = pd.read_html(str(goaliesNR_table))[0]
